In [ ]:
%pip install beacon-api --upgrade


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from beacon_api import *
client = Client("https://beacon-cora-pr.maris.nl", jwt_token="<your_jwt_token_here>")

Connected to: https://beacon-cora-pr.maris.nl/ server successfully


In [3]:
tables = client.list_tables()
tables

{'default': <beacon_api.table.DataTable at 0x7f9b343362a0>}

In [4]:
tables['default'].get_table_schema()

__entry_key: string
PLATFORM_NUMBER: string
PLATFORM_NUMBER._FillValue: string
PLATFORM_NUMBER.long_name: string
STATION_PARAMETERS: string
STATION_PARAMETERS._FillValue: string
STATION_PARAMETERS.long_name: string
STATION_PARAMETERS.conventions: string
DC_REFERENCE: string
DC_REFERENCE._FillValue: string
DC_REFERENCE.long_name: string
DC_REFERENCE.conventions: string
DATA_MODE: string
DATA_MODE._FillValue: string
DATA_MODE.long_name: string
DATA_MODE.conventions: string
WMO_INST_TYPE: string
WMO_INST_TYPE._FillValue: string
WMO_INST_TYPE.long_name: string
WMO_INST_TYPE.conventions: string
TIME: timestamp[ms]
TIME._FillValue: double
TIME.long_name: string
TIME.standard_name: string
TIME.units: string
TIME.valid_min: double
TIME.valid_max: double
TIME.uncertanity: string
TIME.ancillary_variables: string
TIME.calendar: string
TIME.axis: string
TIME_QC: int8
TIME_QC._FillValue: int8
TIME_QC.long_name: string
TIME_QC.conventions: string
TIME_QC.flag_values: string
TIME_QC.valid_min: int8
T

In [ ]:
# METADATA
from datetime import datetime, timezone
dt_str = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")
             
query_builder = tables['default'].query()
query_builder = (
    query_builder
    .add_select_column("__entry_key")
    
    .add_select(SelectLiteral("BEACON_CORA_PR", alias="SOURCE_BDI"))
    .add_select(SelectLiteral("2026-01-12T00:00:00Z", "COMMON_BDI_SNAPSHOT_DATE"))
    .add_select(SelectLiteral("1.5.0", "COMMON_BDI_MONOLITH_VERSION"))
    .add_select(SelectLiteral(dt_str, "COMMON_HARMONIZATION_DATE"))
    
    
    .add_select(SelectColumn("DC_REFERENCE", alias="SOURCE_BDI_DATASET_ID"))
    .add_select(SelectColumn("PLATFORM_NUMBER"))
    .add_select(SelectColumn("DC_REFERENCE"))
    .add_select(SelectColumn("WMO_INST_TYPE"))
    .add_select(SelectColumn("INSTITUTION_EDMO_CODE"))
    .add_select(SelectColumn(".bigram"))
    .add_select(SelectColumn("INSTITUTION_EDMO_CODE", alias="COMMON_EDMO_CODE"))
    .add_select(SelectFunction("concat", [SelectColumn("__entry_key"), SelectColumn("DC_REFERENCE")], "COMMON_ODV_TAG"))
    .add_select(SelectFunction("map_cmems_bigram_l06", [SelectColumn(".bigram"),SelectFunction("btrim",[SelectColumn("WMO_INST_TYPE")])], alias="COMMON_PLATFORM_L06"))
    .add_select(
        SelectFunction("coalesce", 
            [
               SelectFunction("map_c17", [SelectColumn("PLATFORM_NUMBER")]), 
               SelectFunction("map_call_sign_c17", [SelectColumn("PLATFORM_NUMBER"), SelectColumn("TIME")])
            ], 
            alias="COMMON_PLATFORM_C17")
        )
)

In [6]:
query_builder = (
    query_builder
    .add_select(SelectColumn("TIME"))
    .add_select(SelectColumn("TIME_QC"))
    .add_select(SelectColumn("TIME", alias="COMMON_TIME"))
    .add_select(SelectColumn("TIME_QC", alias="COMMON_TIME_QC"))
    
    .add_select(SelectColumn("LATITUDE"))
    .add_select(SelectColumn("LATITUDE.standard_name"))
    .add_select(SelectColumn("LATITUDE.units"))
    .add_select(SelectColumn("LATITUDE", alias="COMMON_LATITUDE"))
    .add_select(SelectLiteral("degrees_north", "COMMON_LATITUDE_UNITS"))
    .add_select(SelectLiteral("Latitude", "COMMON_LATITUDE_STANDARD_NAME"))
    .add_select(SelectLiteral("SDN:P01::ALATZZ01", "COMMON_LATITUDE_P01"))
    .add_select(SelectLiteral("SDN:P06::DEGN", "COMMON_LATITUDE_P06"))
    
    .add_select(SelectColumn("LONGITUDE"))
    .add_select(SelectColumn("LONGITUDE.standard_name"))
    .add_select(SelectColumn("LONGITUDE.units"))
    .add_select(SelectColumn("LONGITUDE", alias="COMMON_LONGITUDE"))
    .add_select(SelectLiteral("degrees_east", "COMMON_LONGITUDE_UNITS"))
    .add_select(SelectLiteral("Longitude", "COMMON_LONGITUDE_STANDARD_NAME"))
    .add_select(SelectLiteral("SDN:P01::ALONZZ01", "COMMON_LONGITUDE_P01"))
    .add_select(SelectLiteral("SDN:P06::DEGE", "COMMON_LONGITUDE_P06"))
    
    .add_select(SelectColumn("POSITION_QC"))
    .add_select(SelectColumn("POSITION_QC", alias="COMMON_POSITION_QC"))
    
    # DEPTH
    
    .add_select_column("PRES")
    .add_select_column("PRES_ADJUSTED")
    .add_select_column("PRES.standard_name")
    .add_select_column("PRES_ADJUSTED.standard_name")
    .add_select_column("PRES.units")
    .add_select_column("PRES_ADJUSTED.units")
    .add_select_column("PRES_QC")
    .add_select_column("PRES_ADJUSTED_QC")
    .add_select_column("DEPH")
    .add_select_column("DEPH_ADJUSTED")
    .add_select_column("DEPH_QC")
    .add_select_column("DEPH_ADJUSTED_QC")
    .add_select_column("DEPH.standard_name")
    .add_select_column("DEPH_ADJUSTED.standard_name")
    .add_select_column("DEPH.units")
    .add_select_column("DEPH_ADJUSTED.units")
    
    # COALESCED DEPTH
    .add_select_coalesced(["PRES_ADJUSTED", "PRES", "DEPH"], "COMMON_ORIGIN_DEPTH")
    .add_select_coalesced(["PRES_ADJUSTED_QC","PRES_QC","DEPH_QC"], "COMMON_ORIGIN_DEPTH_QC")
    .add_select_coalesced(["PRES_ADJUSTED.units", "PRES.units", "DEPH.units"], "COMMON_ORIGIN_DEPTH_UNITS")
    .add_select_coalesced(["PRES_ADJUSTED.standard_name", "PRES.standard_name", "DEPH.standard_name"], "COMMON_ORIGIN_DEPTH_STANDARD_NAME")
    .add_select(SelectFunction("coalesce_label", [SelectColumn("PRES"), SelectLiteral("SDN:P01::PRESPR01"), SelectColumn("DEPH"), SelectLiteral("SDN:P01::ADEPZZ01")], "COMMON_ORIGIN_DEPTH_P01"))
    .add_select(SelectFunction("coalesce_label", [SelectColumn("PRES"), SelectLiteral("SDN:P06::UPDB"), SelectColumn("DEPH"), SelectLiteral("SDN:P06::ULAA")], "COMMON_ORIGIN_DEPTH_P06"))
    
    .add_select(Functions.coalesce([Functions.map_pressure_to_depth(Functions.coalesce(["PRES_ADJUSTED", "PRES"],"pres"), "LATITUDE", "pres_harmonized"), "DEPH"], "COMMON_DEPTH"))
    .add_select_coalesced(["PRES_ADJUSTED_QC","PRES_QC","DEPH_QC"], "COMMON_DEPTH_QC")
    .add_select(SelectLiteral("m", "COMMON_DEPTH_UNITS"))
    .add_select(SelectLiteral("depth", "COMMON_DEPTH_STANDARD_NAME"))
    .add_select(SelectLiteral("SDN:P01::ADEPZZ01", "COMMON_DEPTH_P01"))
    .add_select(SelectLiteral("SDN:P06::ULAA", "COMMON_DEPTH_P06"))
    
)

In [7]:
# map temperature
query_builder = (
    query_builder
    .add_select_column("TEMP")
    .add_select_column("TEMP_QC")
    .add_select_column("TEMP.units")
    .add_select_column("TEMP.standard_name")
    
    .add_select_column("TEMP_ADJUSTED")
    .add_select_column("TEMP_ADJUSTED_QC")
    .add_select_column("TEMP_ADJUSTED.units")
    .add_select_column("TEMP_ADJUSTED.standard_name")
    
    .add_select_coalesced(["TEMP", "TEMP_ADJUSTED"], "COMMON_ORIGIN_TEMPERATURE")
    .add_select_coalesced(["TEMP_QC", "TEMP_ADJUSTED_QC"], "COMMON_ORIGIN_TEMPERATURE_QC")
    .add_select_coalesced(["TEMP.units", "TEMP_ADJUSTED.units"], "COMMON_ORIGIN_TEMPERATURE_UNITS")
    .add_select_coalesced(["TEMP.standard_name", "TEMP_ADJUSTED.standard_name"], "COMMON_ORIGIN_TEMPERATURE_STANDARD_NAME")
    .add_select(SelectLiteral("SDN:P01::TEMPPR01", "COMMON_ORIGIN_TEMPERATURE_P01"))
    .add_select(SelectLiteral("SDN:P06::UPAA", "COMMON_ORIGIN_TEMPERATURE_P06"))
    
    .add_select_coalesced(["TEMP", "TEMP_ADJUSTED"], "COMMON_TEMPERATURE")
    .add_select_coalesced(["TEMP_QC", "TEMP_ADJUSTED_QC"], "COMMON_TEMPERATURE_QC")
    .add_select(SelectLiteral("degree_C", "COMMON_TEMPERATURE_UNITS"))
    .add_select(SelectLiteral("sea_water_temperature", "COMMON_TEMPERATURE_STANDARD_NAME"))
    .add_select(SelectLiteral("SDN:P01::TEMPPR01", "COMMON_TEMPERATURE_P01"))
    .add_select(SelectLiteral("SDN:P06::UPAA", "COMMON_TEMPERATURE_P06"))
    
    # Set instrument type
    .add_select(
        SelectFunction("coalesce", [
            SelectFunction("map_wmo_instrument_type_l05", [SelectFunction("btrim",[SelectColumn("WMO_INST_TYPE")])]),
            SelectFunction("map_l22_l05", [SelectFunction("map_wmo_instrument_type_l22", [SelectFunction("btrim",[SelectColumn("WMO_INST_TYPE")])])]),
            SelectFunction("map_cmems_bigram_l05", [SelectColumn(".bigram")])
            ], "COMMON_TEMPERATURE_L05"))
    .add_select(SelectFunction("map_wmo_instrument_type_l22", [SelectFunction("btrim",[SelectColumn("WMO_INST_TYPE")])], "COMMON_TEMPERATURE_L22"))
)

In [8]:
# map salinity
query_builder = (
    query_builder
    .add_select_column("PSAL")
    .add_select_column("PSAL_QC")
    .add_select_column("PSAL.units")
    .add_select_column("PSAL.standard_name")
    
    .add_select_column("PSAL_ADJUSTED")
    .add_select_column("PSAL_ADJUSTED_QC")
    .add_select_column("PSAL_ADJUSTED.units")
    .add_select_column("PSAL_ADJUSTED.standard_name")
    
    .add_select_coalesced(["PSAL", "PSAL_ADJUSTED"], "COMMON_ORIGIN_SALINITY")
    .add_select_coalesced(["PSAL_QC", "PSAL_ADJUSTED_QC"], "COMMON_ORIGIN_SALINITY_QC")
    .add_select_coalesced(["PSAL.units", "PSAL_ADJUSTED.units"], "COMMON_ORIGIN_SALINITY_UNITS")
    .add_select_coalesced(["PSAL.standard_name", "PSAL_ADJUSTED.standard_name"], "COMMON_ORIGIN_SALINITY_STANDARD_NAME")
    .add_select(SelectLiteral("SDN:P01::PSLTZZ01", "COMMON_ORIGIN_SALINITY_P01"))
    .add_select(SelectLiteral("SDN:P06::UUUU", "COMMON_ORIGIN_SALINITY_P06"))
    
    .add_select_coalesced(["PSAL", "PSAL_ADJUSTED"], "COMMON_SALINITY")
    .add_select_coalesced(["PSAL_QC", "PSAL_ADJUSTED_QC"], "COMMON_SALINITY_QC")
    .add_select(SelectLiteral("sea_water_practical_salinity", "COMMON_SALINITY_STANDARD_NAME"))
    .add_select(SelectLiteral("dimensionless", "COMMON_SALINITY_UNITS"))
    .add_select(SelectLiteral("SDN:P01::PSLTZZ01", "COMMON_SALINITY_P01"))
    .add_select(SelectLiteral("SDN:P06::UUUU", "COMMON_SALINITY_P06"))
    
    # Set instrument type
    .add_select(
        SelectFunction("coalesce", [
            SelectFunction("map_wmo_instrument_type_l05", [SelectFunction("btrim",[SelectColumn("WMO_INST_TYPE")])]),
            SelectFunction("map_l22_l05", [SelectFunction("map_wmo_instrument_type_l22", [SelectFunction("btrim",[SelectColumn("WMO_INST_TYPE")])])]),
            SelectFunction("map_cmems_bigram_l05", [SelectColumn(".bigram")])
            ], "COMMON_SALINITY_L05"))
    .add_select(SelectFunction("map_wmo_instrument_type_l22", [SelectFunction("btrim",[SelectColumn("WMO_INST_TYPE")])], "COMMON_SALINITY_L22"))
)

In [ ]:
import os 
import json

# Make output dir
output_dir = "./output"
os.makedirs(output_dir, exist_ok=True)
# for each year between 1950 and 2025
for year in range(1950, 2026):
    print(f"Processing year {year}")
    query_builder.filters = []
    query_builder = query_builder.add_range_filter("TIME", f"{year}-01-01T00:00:00Z", f"{year}-12-31T23:59:59Z")
    query_builder = (
        query_builder
        .add_filter(OrFilter([
            IsNotNullFilter("TEMP"),
            IsNotNullFilter("PSAL"),
        ]))
    )
    output_file = os.path.join(output_dir, f"cora_pr_{year}.parquet")
    try:
        query_builder.to_parquet(output_file)
        print(f"Saved to {output_file}")
    except Exception as e:
        print(f"Failed to process year {year}: {e}")

Processing year 1950
Running query: {"from": "default", "select": [{"column": "__entry_key", "alias": null}, {"value": "BEACON_CORA_PR", "alias": "SOURCE_BDI"}, {"column": "DC_REFERENCE", "alias": "SOURCE_BDI_DATASET_ID"}, {"column": "PLATFORM_NUMBER", "alias": null}, {"column": "DC_REFERENCE", "alias": null}, {"column": "WMO_INST_TYPE", "alias": null}, {"column": "INSTITUTION_EDMO_CODE", "alias": null}, {"column": ".bigram", "alias": null}, {"column": "INSTITUTION_EDMO_CODE", "alias": "COMMON_EDMO_CODE"}, {"function": "concat", "args": [{"column": "__entry_key", "alias": null}, {"column": "DC_REFERENCE", "alias": null}], "alias": "COMMON_ODV_TAG"}, {"function": "map_cmems_bigram_l06", "args": [{"column": ".bigram", "alias": null}, {"function": "btrim", "args": [{"column": "WMO_INST_TYPE", "alias": null}], "alias": null}], "alias": "COMMON_PLATFORM_L06"}, {"function": "coalesce", "args": [{"function": "map_c17", "args": [{"column": "PLATFORM_NUMBER", "alias": null}], "alias": null}, {"